```
22/02/03
1. 베이스라인 제출(랜덤 포레스트) = 0.639
2. 정규화 후 제출(랜덤 포레스트) = 0.62
3. 이상치 제거 후 제출(랜덤 포레스트) = 0.5933
4. 이상치 제거 + 정규화 후 제출해보기 = 0.535 
5. 정규화 + lightgbm = 0.644 
6. 정규화 + 보팅(xgb,lightgmb,rf) = 0.662

22/02/04
1. 보팅(xgb,lightgmb,rf) = 0.699 (best - 24th)
2. gridsearch + rf = 0.627
3. 하이퍼파라미터 + 보팅 = 0.686
4. 보팅(모델 추가) = 0.676

22/02/07
1. 베이지안 + 파라미터 + 보팅 = 0.597

22/02/13
1. pycaret(fold =5 , top5) = 0.519
```

```
# 변수 설명

index 구분자
quality 품질
fixed acidity 산도
volatile acidity 휘발성산
citric acid 시트르산
residual sugar 잔당 : 발효 후 와인 속에 남아있는 당분
chlorides 염화물
free sulfur dioxide 독립 이산화황
total sulfur dioxide 총 이산화황
density 밀도
pH 수소이온농도
sulphates 황산염
alcohol 도수
type 종류
```

In [1]:
import pandas as pd

import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


# 베이스라인 코드

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(data = train.corr(), annot=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.boxplot(data = train[''])

In [ ]:
plt.figure(figsize=(12,12))
for i in range(1,13):
    plt.subplot(3,4,i)
    sns.distplot(train.iloc[:,i])
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
X_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)

In [ ]:
pred = dt_clf.predict(X_test)

In [ ]:
pred

In [ ]:
dt_clf.feature_importances_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(random_state=0, max_depth=10) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['quality'] = pred

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)

## 전처리 시작 

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


train

In [ ]:
test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


test

In [ ]:
train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)

In [ ]:
y_train = train['quality']

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0, max_depth=10) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

## X_train 이상치 제거, y_train index 맞춤

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
for i in range(len(X_train.columns)-1):
    quantile_25 = np.quantile(X_train[X_train.columns[i]] , 0.25)
    quantile_75 = np.quantile(X_train[X_train.columns[i]] , 0.75)
    IQR = quantile_75 - quantile_25
    minimum = quantile_25 - 1.5 * IQR
    maximum = quantile_75 + 1.5 * IQR
    X_train_outer = X_train[(X_train[X_train.columns[i]] < minimum) | (X_train[X_train.columns[i]] > maximum)]
    X_train = X_train.drop(X_train_outer.index, axis = 0)

In [ ]:
X_train

In [ ]:
y_train = y_train[X_train.index]

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0, max_depth=10) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# X_train 이상치제거 + 정규화

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
for i in range(len(X_train.columns)-1):
    quantile_25 = np.quantile(X_train[X_train.columns[i]] , 0.25)
    quantile_75 = np.quantile(X_train[X_train.columns[i]] , 0.75)
    IQR = quantile_75 - quantile_25
    minimum = quantile_25 - 1.5 * IQR
    maximum = quantile_75 + 1.5 * IQR
    X_train_outer = X_train[(X_train[X_train.columns[i]] < minimum) | (X_train[X_train.columns[i]] > maximum)]
    X_train = X_train.drop(X_train_outer.index, axis = 0)

In [ ]:
X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])

X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_train

In [ ]:
y_train = y_train[X_train.index]

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state=0, max_depth=5) 
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# import lightgbm as lgbm

In [ ]:
import lightgbm as lgbm

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])

X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_test

In [ ]:
model = lgbm.LGBMClassifier()
model.fit(X_train,y_train)

In [ ]:
pred = model.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 보팅

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

## 정규화 후 보팅

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])

X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_train[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])  # 데이터프레임이 되어버림

X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']] = scaler.transform(X_test[['fixed acidity','volatile acidity','citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']])


X_test

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
RF = RandomForestClassifier()

In [ ]:
parameters = {'max_depth':[10], 'min_samples_split':[4]} # 결과를 알고 있음.

In [ ]:
grid_RF = GridSearchCV(RF, param_grid=parameters, cv=3, refit=True, return_train_score=True)
grid_RF.fit(X_train, y_train)

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_RF.best_params_)

In [ ]:
estimator = grid_RF.best_estimator_
estimator

In [ ]:
pred = estimator.predict(X_test)
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 보팅 + 최적의 파라미터

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier(max_depth=19, n_estimators=56)

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 보팅(모델 추가)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [ ]:
X_train = train.drop(['index','quality'], axis = 1)
X_test = test.drop(['index'], axis = 1)
y_train = train['quality']

In [54]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier  

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors=10)

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM),('LR', lr_clf),('KNN', knn_clf)],voting = 'soft')

In [ ]:
VC.fit(X_train,y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 베이지안

### 베이지안 - 랜덤포레스트

In [55]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [56]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [57]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [58]:
from bayes_opt import BayesianOptimization

In [86]:
X = train.drop(columns = ['index', 'quality'])
y = train['quality']

In [87]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size = 0.2, random_state = 777)

In [88]:
rf_parameter_bounds = {
                      'max_depth' : (1,4), # 나무의 깊이
                      'n_estimators' : (30,100),
                      }

In [89]:
def rf_bo(max_depth, n_estimators):
  rf_params = {
              'max_depth' : int(round(max_depth)),
               'n_estimators' : int(round(n_estimators)),      
              }
  rf = RandomForestClassifier(**rf_params)


  rf.fit(X_train,y_train)
  score = accuracy_score(y_valid, rf.predict(X_valid))
  return score

In [90]:
BO_rf = BayesianOptimization(f = rf_bo, pbounds = rf_parameter_bounds,random_state = 0)
# pbounds : 하이퍼 파라미터 값의 최소~최대 지정

In [91]:
BO_rf.maximize(init_points = 5, n_iter = 5)
# init_points : 몇 번 탐색할지
# n_iter : 최적 값을 몇 번 찾아 갈지
# maximize : 반복하며 가장 높은 target값을 찾는다

|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.5409   |  2.646    |  80.06    |
|  2        |  0.5436   |  2.808    |  68.14    |
|  3        |  0.5382   |  2.271    |  75.21    |
|  4        |  0.5455   |  2.313    |  92.42    |
|  5        |  0.5636   |  3.891    |  56.84    |
|  6        |  0.5636   |  4.0      |  47.41    |
|  7        |  0.4627   |  1.0      |  30.0     |
|  8        |  0.4627   |  1.0      |  51.89    |
|  9        |  0.56     |  3.956    |  47.36    |
|  10       |  0.5591   |  3.985    |  48.53    |


In [92]:
max_params = BO_rf.max['params']

max_params['max_depth'] = int(max_params['max_depth'])
max_params['n_estimators'] = int(max_params['n_estimators'])
print(max_params)

{'max_depth': 3, 'n_estimators': 56}


### 베이지안 - XGBoost

In [122]:
xgb_parameter_bounds = {
                      'gamma' : (0,10),
                      'max_depth' : (1,4), 
                      'subsample' : (0.5,1)
                      }

In [123]:
def xgb_bo(gamma,max_depth, subsample):

  xgb_params = {
              'gamma' : int(round(gamma)),
              'max_depth' : int(round(max_depth)),
               'subsample' : int(round(subsample)),      
              }

  xgb = XGBClassifier(**xgb_params)
    

  xgb.fit(X_train,y_train)
  score = accuracy_score(y_valid, xgb.predict(X_valid))
  return score

In [124]:
BO_xgb = BayesianOptimization(f = xgb_bo, pbounds = xgb_parameter_bounds,random_state = 0)

In [125]:
BO_xgb.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   |   gamma   | max_depth | subsample |
-------------------------------------------------------------
[14:42:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  1        |  0.5745   |  5.488    |  3.146    |  0.8014   |
[14:42:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
|  2        |  0.5709   |  5.449    |  2.271    |  0.8229   |
[14:42:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, t

In [126]:
max_params = BO_xgb.max['params']
max_params

{'gamma': 5.680445610939323,
 'max_depth': 3.776789914877983,
 'subsample': 0.5355180290989434}

### 베이지안 - LGBM

In [127]:
lgbm_parameter_bounds = {
                      'n_estimators' : (30,100),
                      'max_depth' : (1,4), # 나무의 깊이
                      'subsample' : (0.5,1)
                      }

In [128]:
def lgbm_bo(n_estimators,max_depth, subsample):

  lgbm_params = {
              'n_estimators' : int(round(n_estimators)),
              'max_depth' : int(round(max_depth)),
               'subsample' : int(round(subsample)),      
              }

  lgbm = LGBMClassifier(**lgbm_params)



  lgbm.fit(X_train,y_train)
  score = accuracy_score(y_valid, lgbm.predict(X_valid))
  return score

In [129]:
BO_lgbm = BayesianOptimization(f = lgbm_bo, pbounds = lgbm_parameter_bounds,random_state = 0)

In [130]:
BO_lgbm.maximize(init_points = 5, n_iter = 5)

|   iter    |  target   | max_depth | n_esti... | subsample |
-------------------------------------------------------------
|  1        |  0.59     |  2.646    |  80.06    |  0.8014   |
|  2        |  0.5927   |  2.635    |  59.66    |  0.8229   |
|  3        |  0.5855   |  2.313    |  92.42    |  0.9818   |
|  4        |  0.5791   |  2.15     |  85.42    |  0.7644   |
|  5        |  0.5891   |  2.704    |  94.79    |  0.5355   |
|  6        |  0.59     |  3.008    |  79.72    |  0.9232   |
|  7        |  0.5818   |  2.185    |  64.04    |  0.5565   |
|  8        |  0.5945   |  2.768    |  57.29    |  0.8886   |


LightGBMError: Check failed: (bagging_fraction) > (0.0) at D:\a\1\s\python-package\compile\src\io\config_auto.cpp, line 356 .


In [131]:
max_params = BO_lgbm.max['params']
max_params

{'max_depth': 2.76817440872047,
 'n_estimators': 57.28536678996605,
 'subsample': 0.888619690346877}

In [132]:
LGBM = LGBMClassifier(max_depth = 3,n_estimators=57, subsample = 0.8886)
XGB = XGBClassifier(gamma =  5.6804, max_depth = 3, subsample = 0.5355)
RF = RandomForestClassifier(max_depth = 3, n_estimators = 56)

# VotingClassifier 정의
VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [133]:
VC.fit(X,y)

[14:47:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=3,
                                                     n_estimators=56)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            enable_categorical=False,
                                            gamma=5.6804, gpu_id=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_de...
                                            min_child_weight=None, missing=nan,
            

In [134]:
X_test = test.drop(['index'], axis = 1)

In [135]:
pred = VC.predict(X_test)

In [136]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = pred

submission

submission.to_csv('submission.csv',index=False)

# Pycaret

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(train['type'])
train['type'] = enc.transform(train['type'])
test['type'] = enc.transform(test['type'])

In [4]:
train = train.drop('index', axis = 1)
test = test.drop('index', axis = 1)

In [5]:
train

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,1
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,0
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,1
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,1
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,5,7.7,0.150,0.29,1.3,0.029,10.0,64.0,0.99320,3.35,0.39,10.1,1
5493,6,6.3,0.180,0.36,1.2,0.034,26.0,111.0,0.99074,3.16,0.51,11.0,1
5494,7,7.8,0.150,0.34,1.1,0.035,31.0,93.0,0.99096,3.07,0.72,11.3,1
5495,5,6.6,0.410,0.31,1.6,0.042,18.0,101.0,0.99195,3.13,0.41,10.5,1


In [6]:
from pycaret.classification import *

In [7]:
setup_clf = setup(data = train, target = 'quality', fold_shuffle=True)

,Description,Value
0,session_id,3174
1,Target,quality
2,Target Type,Multiclass
3,Label Encoded,"3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6"
4,Original Data,"(5497, 13)"
5,Missing Values,False
6,Numeric Features,12
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [18]:
top2 = compare_models(sort='Accuracy', n_select=5, fold = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.6384,0.3233,0.3674,0.6486,0.6244,0.4326,0.4385,0.1420
et,Extra Trees Classifier,0.6311,0.3253,0.3682,0.6399,0.6176,0.4211,0.4272,0.1220
xgboost,Extreme Gradient Boosting,0.6189,0.3175,0.3629,0.6166,0.6074,0.4089,0.4122,0.6620
catboost,CatBoost Classifier,0.6171,0.3181,0.3674,0.6161,0.6063,0.4060,0.4092,3.7860
lightgbm,Light Gradient Boosting Machine,0.6005,0.3166,0.3408,0.6030,0.5919,0.3871,0.3905,0.3920
gbc,Gradient Boosting Classifier,0.5701,0.3018,0.3074,0.5616,0.5553,0.3269,0.3320,0.8300
dt,Decision Tree Classifier,0.5583,0.2722,0.3460,0.5564,0.5565,0.3407,0.3411,0.0100
lda,Linear Discriminant Analysis,0.5334,0.2863,0.2651,0.5272,0.5115,0.2601,0.2667,0.0100
ridge,Ridge Classifier,0.5295,0.0000,0.2167,0.5024,0.4624,0.2188,0.2381,0.0100
lr,Logistic Regression,0.5274,0.2758,0.2248,0.4871,0.4820,0.2267,0.2406,0.3180


In [19]:
tuned_top5 = [tune_model(i, fold = 5) for i in top2]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5974,0.7714,0.3155,0.5985,0.5858,0.3714,0.3764
1,0.5844,0.7602,0.2774,0.5727,0.5672,0.3483,0.3532
2,0.5787,0.0000,0.3174,0.5575,0.5639,0.3471,0.3493
3,0.5722,0.0000,0.3494,0.5635,0.5637,0.3418,0.3434
4,0.5787,0.0000,0.3423,0.5658,0.5640,0.3446,0.3475
Mean,0.5823,0.3063,0.3204,0.5716,0.5689,0.3507,0.3540
SD,0.0085,0.3752,0.0253,0.0143,0.0085,0.0106,0.0117


In [17]:
blender_top5 = blend_models(estimator_list=tuned_top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.5351,0.0000,0.2252,0.4363,0.4578,0.2112,0.2546
1,0.5455,0.0000,0.2286,0.4345,0.4659,0.2290,0.2693
2,0.4961,0.0000,0.2090,0.3850,0.4237,0.1528,0.1736
3,0.5377,0.0000,0.2245,0.4400,0.4578,0.2133,0.2608
4,0.5299,0.0000,0.2232,0.4248,0.4545,0.2051,0.2402
5,0.5091,0.7280,0.1845,0.4001,0.4354,0.1733,0.1996
6,0.5455,0.7184,0.1970,0.4447,0.4674,0.2293,0.2752
7,0.5026,0.0000,0.2065,0.4169,0.4178,0.1459,0.1919
8,0.5156,0.0000,0.2166,0.4099,0.4403,0.1801,0.2113
9,0.5234,0.0000,0.2172,0.4322,0.4421,0.1853,0.2335


In [20]:
final_model = finalize_model(blender_top5)
prediction = predict_model(final_model, data= test)

In [21]:
prediction['Label']

0      6
1      5
2      6
3      6
4      6
      ..
995    5
996    6
997    5
998    6
999    6
Name: Label, Length: 1000, dtype: int32

In [22]:
submission = pd.read_csv('sample_submission.csv')

submission['quality'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)